<a href="https://colab.research.google.com/github/BlitzKriegM/C-digos-Proyecto/blob/main/Dataset_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [ ]:
# Installl dependencies
!pip install -q openai
!pip install -q langchain
!pip install -q datasets argilla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 25.9 M

## Import libraries

In [ ]:
# Import libraries
import numpy as np
import pandas as pd




### Parameters definition

In [ ]:
# Define parameters
rac_directory_path = "/content/drive/MyDrive/FULL-Seminario/Proyectos/rac_gpt/racs"





## PDF Reader

## Question answering generation

# Connect with argilla


In [ ]:
from datasets import load_dataset

import argilla as rg

# connect to your app endpoint (uses default team API key)
rg.init(api_url="https://ejbejaranos-rac-gpt-dataset-v0.hf.space", api_key="123456789")

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:182: UserWarning: You're connecting to Argilla Server 1.13.0 using a different client version (1.18.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [ ]:
# Argilla dataset configuration
#TODO: CHECK
dataset = rg.FeedbackDataset(
    guidelines="Please, read the question carefully and try to answer it as accurately as possible.",
    fields=[
        rg.TextField(name="question", required=True),
        rg.TextField(name="answer", required=True, use_markdown=True),
    ],
    questions=[
        rg.RatingQuestion(
            name="question_quality",
            description="How would you rate the quality of the question?",
            values=[1, 2, 3, 4, 5],
        ),
        rg.LabelQuestion(
            name="relevant",
            title="Es este sample verdader?",
            labels={"YES": "Yes", "NO": "No"}, # or ["YES","NO"]
            required=True,
            visible_labels=None
        ),
        rg.TextQuestion(
            name="answer_correction",
            description="If you think the answer is not accurate, please, correct it.",
            required=False,
        ),
    ]
)

## To upload only one record first time

In [ ]:
# create a single Feedback Record
record = rg.FeedbackRecord(
    fields={
        "question": "Que es el RAC?",
        "answer": "El RAC ES rock, abrazos y comida."
    },
    metadata={"source": "RAC01"},
    external_id=None
)

dataset.add_records(record)
dataset.push_to_argilla(name="Unrecord", workspace="admin")

Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00, 12.74it/s]


RemoteFeedbackDataset(
   id=d7e92abd-2d16-40e8-b0e5-fadcf29698f2
   name=Unrecord
   workspace=Workspace(id=1a5d549c-a95a-4ec3-a798-47b17dc431ba, name=admin, inserted_at=2023-11-04 21:21:12, updated_at=2023-11-04 21:21:12)
   url=https://ejbejaranos-rac-gpt-dataset-v0.hf.space/dataset/d7e92abd-2d16-40e8-b0e5-fadcf29698f2/annotation-mode
   fields=[RemoteTextField(id=UUID('f546c954-490c-4568-a27a-d81b66bd60ae'), client=None, name='question', title='Question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('fd748666-db9e-45db-bc48-6eebac75323c'), client=None, name='answer', title='Answer', required=True, type='text', use_markdown=True)]
   questions=[RemoteRatingQuestion(id=UUID('dc7d038e-98a8-44f5-949d-91c240adbddb'), client=None, name='question_quality', title='Question_quality', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5]), RemoteLabelQuestion(id=UUID('5857f485-0b65-45fe-bfb9-227e50eb99e2'), client=None, name='relevant', title='Es 

## To upload all dataset first time

In [ ]:
# Feedback Records as a list of dictionaries
data_random = [
    {
        "question": "What is the capital of France?",
        "answer": "The capital of France is Paris.",
    },
    {
        "question": "Who wrote the play 'Romeo and Juliet'?",
        "answer": "The play 'Romeo and Juliet' was written by William Shakespeare.",
    },
    {
        "question": "What is the chemical symbol for water?",
        "answer": "The chemical symbol for water is H2O.",
    },
    {
        "question": "When did the first moon landing occur?",
        "answer": "The first moon landing occurred on July 20, 1969.",
    }
]
records = [rg.FeedbackRecord(fields={"question": record["question"], "answer": record["answer"]})  for record in data_random]
# For multiple records

dataset.add_records(records)
dataset.push_to_argilla(name="multiplerecord2", workspace="admin")

Pushing records to Argilla...: 100%|██████████| 1/1 [00:00<00:00, 12.11it/s]


RemoteFeedbackDataset(
   id=ae262745-b805-469f-9c2d-bb85cefe9d6c
   name=multiplerecord2
   workspace=Workspace(id=1a5d549c-a95a-4ec3-a798-47b17dc431ba, name=admin, inserted_at=2023-11-04 21:21:12, updated_at=2023-11-04 21:21:12)
   url=https://ejbejaranos-rac-gpt-dataset-v0.hf.space/dataset/ae262745-b805-469f-9c2d-bb85cefe9d6c/annotation-mode
   fields=[RemoteTextField(id=UUID('646f6c8e-77ed-4698-b7ac-e6e93ac6382f'), client=None, name='question', title='Question', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('18d205da-90f6-4153-b8f3-8c01d78d3be1'), client=None, name='answer', title='Answer', required=True, type='text', use_markdown=True)]
   questions=[RemoteRatingQuestion(id=UUID('b534ef49-a81b-4783-bd50-700de76d13ee'), client=None, name='question_quality', title='Question_quality', description=None, required=True, type='rating', values=[1, 2, 3, 4, 5]), RemoteLabelQuestion(id=UUID('eb8ea6ba-7f0a-416a-83b5-a303e88bc091'), client=None, name='relevant', tit

### To add new samples in the same dataset

In [ ]:
# Feedback Records as a list of dictionaries
data_random = [
    {
        "question": "What is the capital of Colombia?",
        "answer": "The capital of Colombia is Bogotá.",
    },
    {
        "question": "Which famous river runs through Colombia?",
        "answer": "The Magdalena River runs through Colombia.",
    },
    {
        "question": "What is the official language of Colombia?",
        "answer": "The official language of Colombia is Spanish.",
    },
    {
        "question": "What are the famous coffee regions in Colombia?",
        "answer": "Some of the famous coffee regions in Colombia are Quindío, Risaralda, and Caldas.",
    },
    {
        "question": "What is the name of the tallest mountain in Colombia?",
        "answer": "The tallest mountain in Colombia is the Nevado del Huila.",
    },
    {
        "question": "Which Colombian city is known for its street art?",
        "answer": "Bogotá is known for its vibrant street art scene.",
    },
    {
        "question": "What is the traditional Colombian dish called?",
        "answer": "The traditional Colombian dish is called Bandeja Paisa.",
    },
    {
        "question": "Which famous Colombian writer won the Nobel Prize in Literature?",
        "answer": "Gabriel García Márquez won the Nobel Prize in Literature.",
    },
    {
        "question": "What are the famous archaeological ruins in Colombia?",
        "answer": "The famous archaeological ruins in Colombia are at Ciudad Perdida (Lost City).",
    },
    {
        "question": "Which colorful river in Colombia is often called 'The River of Five Colors'?",
        "answer": "Caño Cristales is often called 'The River of Five Colors'.",
    }
]


records = [rg.FeedbackRecord(fields={"question": record["question"], "answer": record["answer"]})  for record in data_random]
# For multiple records

dataset.add_records(records)
dataset.push_to_argilla()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 50>:50                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: ArgillaMixin.push_to_argilla() missing 1 required positional argument: 'name'